## 1. 환경 설정

In [3]:
from re import search

# (1) Env 환경변수
from dotenv import load_dotenv

load_dotenv('../.env')

# (2) 기본 라이브러리
import os
from glob import glob

from pprint import pprint

# 벡터저장소 로드
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
)

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


벡터 저장소에 저장된 문서 수: 5


## 2. LangChain LCEL
### 2.1 Prompt + LLM

In [4]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.3,
    max_tokens=100,
)

messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

# 템플릿 입력 변수를 출력
print(prompt.input_variables)

# input 값을 전달하여 프롬프트를 랜더링
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")
print(prompt_text)

# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)
print(response.content)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]
['query']
System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?
테슬라의 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타페니스(Marc Tarpenning), 그리고 이반 로버츠(Ivan Rodriguez)입니다. 특히 일론 머스크는 초기 투자자이자 현재 CEO로서 테슬라의 대표적인 인물입니다.


In [5]:
# LCEL 체인을 구성
chain = prompt | llm
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11a01c650>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11a1c3490>, root_client=<openai.OpenAI object at 0x11a18a310>, root_async_client=<openai.AsyncOpenAI object at 0x11a1c1dd0>, model_name='gpt-4.1-nano', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [6]:
# 체인의 입력 스키마를 출력
from pprint import pprint

pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [7]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query": "테슬라 창업자는 누구인가요?"})
print(response.content)

테슬라의 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타페니스(Marc Tarpenning), 그리고 이반 체바(Ivan Chev)입니다. 이 중 일론 머스크는 종종 테슬라의 대표적 인물로 알려져 있으며, 현재도 회사의 중요한 리더입니다.


In [8]:
# 체인을 실행 - 옵션 2
response = chain.invoke("테슬라 창업자는 누구인가요?")
print(response.content)

테슬라의 공동 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다. 이후 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 그리고 이반스 리드(Evan Williams) 등이 회사의 중요한 인물로 참여하며 성장시켰습니다. 특히 일론 머스크는 테슬라의 대표이사로서 회사의 방향성을 이끄


## 2.2 Prompt + LLM + Output Parser
### a) 문자열 파싱 - StrOutputParser

In [9]:

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(response)

str_chain = prompt | llm | output_parser
query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)
print(str_response)

리비안(Rivian)은 2009년에 설립되었습니다.


### b) JSON 출력 - JsonOutputParser

In [10]:
from langchain_core.output_parsers import JsonOutputParser

# 출력 파서를 생성
json_parser = JsonOutputParser()

# 체인을 실행(Json 출력)
json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON형식으로 출력해주세요.")
print(json_response)

json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": "마틴 에버하드, 마크 타페닝, JB 스트라우벨"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 34, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'finish_reason': 'stop', 'logprobs': None} id='run-4531c75c-dea3-49d7-bec0-67fbae07e732-0' usage_metadata={'input_tokens': 34, 'output_tokens': 31, 'total_tokens': 65}
{'창업자': '마틴 에버하드, 마크 타페닝, JB 스트라우벨'}


### c) Schema 지정 - Pydantic

In [11]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator


# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")


# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person)
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions())
print("========================================")

# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")

# 체인을 구성
person_chain = prompt | llm | person_parser
response = person_chain.invoke("테슬라 창업자는 누구인가요?")
print(response)



PydanticOutputParser 프롬프트
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "title": {"title": "Title", "description": "The title or position of the person.", "type": "string"}}, "required": ["name", "title"]}
```
Prompt 템플릿
----------------------------------------
System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema

## 3. Chat Completion Methods
### (1) stream
- 입력에 대한 응답을 실시간 스트림을 생성하여 전달

In [12]:
import time

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    # print 함수는 출력할 때마다 줄바꿈을 하지만, 줄바꿈 없이 하려면 end=""를 사용
    # flush=Ture 옵션을 사용하여 출력 버퍼를 즉시 비움. 데이터를 지연 없이 즉시 출력하는데 유용
    print(chunk.content, end="", flush=True)
    time.sleep(0.1)  # 0.1초 대기(100ms)

테슬라의 공동 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타펜닝(Marc Tarpenning), 그리고 이반스 라이트(Ivan Sutherland)입니다. 특히 일론 머스크는 테슬라의 초기 투자자이자 현재 CEO로서 가장 잘 알려져 있습니다.

### (2) batch
- 입력 리스트에 대한 응답을 배치 단위로 생성

In [13]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?"
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타페니스(Marc Tarpenning), 그리고 이반 차베스(Ivan Chavez)입니다. 특히 일론 머스크는 테슬라의 중요한 초기 투자자이자 CEO로서 큰 역할을 했습니다.

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ Scaringe입니다. 그는 2009년에 리비안을 설립하였으며, 전기 픽업트럭과 SUV를 개발하는 회사로 알려져 있습니다.



## 4. Runnable
### (1) RunnableParallel

In [14]:
# 문서 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 1},
)
query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)
retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])
pprint(retrieved_docs_text)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [15]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

# RunnableParallel 구성
runnable = RunnableParallel(
    {"context": itemgetter("context"),
     "question": itemgetter("question"),}
)

response = runnable.invoke({"context": retrieved_docs_text,
                            "question": query})
print(response)

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.', 'question': '테슬라 창업자는 누구인가요?'}


### (2) RunnablePassthrough

In [16]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnableParallel(
    question = RunnablePassthrough()
)
runnable.invoke("테슬라 창업자는 누구인가요?")

{'question': '테슬라 창업자는 누구인가요?'}

### (3) RunnableLambda
- 정의 : 파이썬의 커스텀 함수를 매핑하는데 사용

In [17]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question=RunnablePassthrough(),
    word_count=RunnableLambda(count_num_words)
)
runnable.invoke("테슬라 창업자는 누구인가요?")

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## 5. 전체 RAG 파이프라인 구성
### (1) RAG 프롬프트 템플릿

In [18]:
# Prompt 템플릿을 생성
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
Do not use any external information or knowledge.
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question]
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿을 출력
prompt.pretty_print()

================================ Human Message =================================

Answer the question based only on the following context.
Do not use any external information or knowledge.
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question]
{question}

[Answer]



### (2) Retriever Chain 연결

In [19]:
# 벡터 검색기
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# 문서 포맷터 함수
def format_docs(docs):
    return  "\n\n".join([d.page_content for d in docs])

# 체인 구성
retriever_chain = retriever | format_docs

# 체인을 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.')


### (3) RAG Chain 연결

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 모델 생성
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    max_tokens=100,
)

# 체인 생성
rag_chain = (
    {"context": retriever_chain,
     "question": RunnablePassthrough()} #runnableParallel
    | prompt
    | llm
    | StrOutputParser()
)

query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke(query)

print(response)

테슬라의 창업자는 마틴 에버하드(CEO)와 마크 타페닝(CFO)입니다.


## 6. Gradio 챗봇
### (1) invoke 실행

In [23]:
import gradio as gr
def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

# Graiio 인터페이스 생성
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

(2) stream 실행

In [ ]:
import gradio as gr

def answer_invoke(message, history):
    partial_message = ""
    for chunk in rag_chain.stream(message):
        if chunk is not None:
            partial_message = partial_message + chunk
            time.sleep(0.1)
            yield partial_message

# Graiio 인터페이스 생성
demo = gr.ChatInterface(fn=answer_invoke(), title="QA Bot")